# Homework for Module 1

Instead of doing SQL scripts, I thought it would be convenient to put it in one notebook, so I am using pandas to query the data.

In [16]:
# Necessary imports
import pandas as pd
import os
from sqlalchemy import create_engine,text

## Question 1. Understanding docker first run
> Run docker with the python:3.12.8 image in an interactive mode, use the entrypoint bash.
> What's the version of pip in the image?

- 24.3.1
- 24.2.1
- 23.3.1
- 23.2.1

In [33]:
# running it through the notebook to have proof for the homework
import subprocess

result = subprocess.run(['docker', 'run', '--rm', 'python:3.12.8', 'pip', '--version'],
                        capture_output=True, text=True)

# Print the result
print(result.stdout)

pip 24.3.1 from /usr/local/lib/python3.12/site-packages/pip (python 3.12)



## Question 2. Understanding Docker networking and docker-compose

In [ ]:
# this is tricky to put here, but I was able to connect using both postgres:5432 and db:5432. I will go with db:5432.

## Connecting to database through SQLAlchemy

In [17]:
# Best practices would be to put these values into environment variables or query secrets manager

# user=os.environ['POSTGRES_USER']
# password=os.environ['POSTGRES_PASSWORD']
# host=os.environ['POSTGRES_HOST']

# But since we are providing docker-compose.yml file with these values, it does not really matter
user='postgres'
password='postgres'
host='localhost'
port=5433
database='ny_taxi'

In [18]:
# Creating engine - connect is not necessary, but it lets us verify the connection
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
engine.connect()

In [21]:
# Lazy way is to query all the data with necessary joins. It works fast for me, but if we have larger dataset and structured report, we can optimize to only use columns needed for the report
query = '''
SELECT gtt.*,
       puz."Borough" as pu_borough,
       puz."Zone" as pu_zone,
       doz."Borough" AS do_borough,
       doz."Zone" AS do_zone
FROM green_taxi_trips gtt
         JOIN zones puz ON gtt."PULocationID" = puz."LocationID"
         JOIN zones doz ON gtt."DOLocationID" = doz."LocationID"
'''
df =pd.read_sql(query,engine)

## Question 3. Trip Segmentation Count
> During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

>  - Up to 1 mile
>  - In between 1 (exclusive) and 3 miles (inclusive),
>  - In between 3 (exclusive) and 7 miles (inclusive),
>  - In between 7 (exclusive) and 10 miles (inclusive),
>  - Over 10 miles

Pandas has a handy way to cut the dataset into specified bins, they do not have to be uniform. It is perfect for such categorizations and much more concise than CASE statements (although some may argue that not as easy to interpret).

In [22]:
bins = [df['trip_distance'].min(),1,3,7,10,df['trip_distance'].max()]
df['trip_distance_category'] = pd.cut(df['trip_distance'], bins=bins,include_lowest=True)

In [24]:
df[
    (df['lpep_pickup_datetime']>='2019-10-01') &
    (df['lpep_dropoff_datetime']<'2019-11-01')
].groupby('trip_distance_category',observed=True).size()

trip_distance_category
(-6.931, 1.0]     104802
(1.0, 3.0]        198924
(3.0, 7.0]        109603
(7.0, 10.0]        27678
(10.0, 515.89]     35189
dtype: int64

## Question 4. Longest trip for each day

> Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

> Tip: For every day, we only care about one single trip with the longest distance.

- 2019-10-11
- 2019-10-24
- 2019-10-26
- 2019-10-31

In [25]:
df['trip_pickup_day'] =(pd.to_datetime(df['lpep_pickup_datetime']).dt.strftime('%Y-%m-%d'))

In [26]:
df.groupby('trip_pickup_day')['trip_distance'].max().sort_values(ascending=False).nlargest(5)

trip_pickup_day
2019-10-31    515.89
2019-10-11     95.78
2019-10-26     91.56
2019-10-24     90.75
2019-10-05     85.23
Name: trip_distance, dtype: float64

## Question 5. Three biggest pickup zones
> Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

> Consider only lpep_pickup_datetime when filtering by date.

- East Harlem North, East Harlem South, Morningside Heights
- East Harlem North, Morningside Heights
- Morningside Heights, Astoria Park, East Harlem South
- Bedford, East Harlem North, Astoria Park

In [27]:
df[df['trip_pickup_day']=='2019-10-18'].groupby('pu_zone')['total_amount'].sum().sort_values(ascending=False)

pu_zone
East Harlem North                   18686.68
East Harlem South                   16797.26
Morningside Heights                 13029.79
Central Harlem                      12440.66
Elmhurst                            12431.96
                                      ...   
Heartland Village/Todt Hill            24.80
Inwood Hill Park                       24.03
Forest Park/Highland Park              21.30
Pelham Bay Park                        12.80
Saint Michaels Cemetery/Woodside        7.60
Name: total_amount, Length: 234, dtype: float64

## Question 6. Largest tip

> For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?

> Note: it's tip , not trip

> We need the name of the zone, not the ID.

- Yorkville West
- JFK Airport
- East Harlem North
- East Harlem South

In [29]:
df[
    (df['lpep_pickup_datetime']>='2019-10-01') &
    (df['lpep_pickup_datetime']<'2019-11-01') &
    (df['pu_zone']=='East Harlem North')
].groupby('do_zone')['tip_amount'].max().sort_values(ascending=False).nlargest(5)

do_zone
JFK Airport              87.30
Yorkville West           80.88
East Harlem North        40.00
Newark Airport           26.45
Upper East Side North    18.45
Name: tip_amount, dtype: float64